In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import MDS
from sklearn.metrics import pairwise_distances
import pandas as pd
from tqdm import tqdm
import os

# Import project-specific modules
from data_structures import PatientData
from decoders import ConceptDecoder, SingleResultsManager
from multi_decoding import MultiResultsManager, plot_multi_patient_heatmap

In [2]:
p562 = PatientData(pid='562')
p563 = PatientData(pid='563')
p566 = PatientData(pid='566')


THRESHOLD = 0.1  # fr threshold


./Data/40m_act_24_S06E01_30fps_character_frames.csv
./Data/40m_act_24_S06E01_30fps_character_frames.csv
./Data/40m_act_24_S06E01_30fps_character_frames.csv


In [3]:
p562_fr_neurons = p562.filter_neurons_by_fr(
    neurons=p562.neurons, 
    window=(p562.times_dict['movie_start_rel'], p562.times_dict['preSleep_recall_start_rel']), 
    threshold=THRESHOLD
)
p562_mtl_fr_neurons = p562.filter_mtl_neurons(neurons=p562_fr_neurons)

# Patient 563
p563_fr_neurons = p563.filter_neurons_by_fr(
    neurons=p563.neurons, 
    window=(p563.times_dict['movie_start_rel'], p563.times_dict['preSleep_recall_start_rel']), 
    threshold=THRESHOLD
)
p563_mtl_fr_neurons = p563.filter_mtl_neurons(neurons=p563_fr_neurons)

# Patient 566
p566_fr_neurons = p566.filter_neurons_by_fr(
    neurons=p566.neurons, 
    window=(p566.times_dict['movie_start_rel'], p566.times_dict['preSleep_recall_start_rel']), 
    threshold=THRESHOLD
)
p566_mtl_fr_neurons = p566.filter_mtl_neurons(neurons=p566_fr_neurons)

# Create lists for multi-patient analysis
neurons_list = [p562_mtl_fr_neurons, p563_mtl_fr_neurons, p566_mtl_fr_neurons]
patient_data_list = [p562, p563, p566]


In [4]:
print(f"Patient 562: {len(p562_mtl_fr_neurons)} MTL neurons (from {len(p562_fr_neurons)} active neurons)")
print(f"Patient 563: {len(p563_mtl_fr_neurons)} MTL neurons (from {len(p563_fr_neurons)} active neurons)")
print(f"Patient 566: {len(p566_mtl_fr_neurons)} MTL neurons (from {len(p566_fr_neurons)} active neurons)")


Patient 562: 3 MTL neurons (from 89 active neurons)
Patient 563: 64 MTL neurons (from 111 active neurons)
Patient 566: 18 MTL neurons (from 75 active neurons)


In [5]:
selected_concepts = [
    'A.Amar',
    'A.Fayed',
    'B.Buchanan',
    'C.Manning',
    'C.OBrian',
    'J.Bauer',
    'K.Hayes',
    'M.OBrian',
    'N.Yassir',
    'R.Wallace',
    'T.Lennox',
]


In [6]:
concept_pairs_to_decode = []
for i, concept1 in enumerate(selected_concepts):
    for concept2 in selected_concepts[i+1:]:  # Avoid duplicates and self-pairs
        concept_pairs_to_decode.append((concept1, concept2))

print(f"Number of concept pairs to decode: {len(concept_pairs_to_decode)}")


Number of concept pairs to decode: 55


In [11]:
multi_mtl_manager = MultiResultsManager(
    patient_data_list=patient_data_list,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=True,  # Use pseudopopulations to balance dataset
    neurons_list=neurons_list
)

# Run decoding with multiple iterations for robustness
num_iterations = 3  # Can increase for more stable results
multi_mtl_manager.run_decoding_for_pairs(num_iter=num_iterations)


Decoding for ['562', '563', '566']:   2%|▏         | 1/55 [00:03<03:22,  3.75s/it]

Skipping concept pair A.Amar, A.Fayed: Insufficient samples for A.Amar vs A.Fayed


Decoding for ['562', '563', '566']:   7%|▋         | 4/55 [00:37<07:44,  9.11s/it]

Skipping concept pair A.Amar, C.OBrian: Insufficient samples for A.Amar vs C.OBrian


Decoding for ['562', '563', '566']:  13%|█▎        | 7/55 [01:13<07:52,  9.85s/it]

...

/Users/ismaelrobles-razzaq/Desktop/research/.venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
Decoding for ['562', '563', '566']:  64%|██████▎   | 35/55 [08:08<05:16, 15.85s/it]/Users/ismaelrobles-razzaq/Desktop/research/.venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
Decoding for ['562', '563', '566']:  67%|██████▋   | 37/55 [08:28<03:42, 12.38s/it]

Skipping concept pair C.OBrian, M.OBrian: Insufficient samples for C.OBrian vs M.OBrian


Decoding for ['562', '563', '566']:  85%|████████▌ | 47/55 [11:17<02:14, 16.78s/it]/Users/ismaelrobles-razzaq/Desktop/research/.venv/lib/python3.11/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
Decoding for ['562', '563', '566']:  89%|████████▉ | 49/55 [11:35<01:14, 12.43s/it]

Skipping concept pair K.Hayes, T.Lennox: Insufficient samples for K.Hayes vs T.Lennox


Decoding for ['562', '563', '566']: 100%|██████████| 55/55 [13:10<00:00, 14.38s/it]


In [21]:
patient_data_list=[p562]

In [22]:
multi_mtl_no_pseudo = MultiResultsManager(patient_data_list=patient_data_list,
    concept_pairs=concept_pairs_to_decode,
    epoch='movie',
    standardize=False,
    pseudo=False,  # Use pseudopopulations to balance dataset
    neurons_list=neurons_list
)


In [23]:

multi_mtl_no_pseudo.run_decoding_for_pairs(1)

Decoding for ['562']:   2%|▏         | 1/55 [00:01<00:56,  1.04s/it]

Skipping concept pair A.Amar, A.Fayed: Insufficient samples for A.Amar vs A.Fayed


Decoding for ['562']:   7%|▋         | 4/55 [00:06<01:18,  1.54s/it]

Skipping concept pair A.Amar, C.OBrian: Insufficient samples for A.Amar vs C.OBrian


Decoding for ['562']:  13%|█▎        | 7/55 [00:11<01:16,  1.59s/it]

...

Decoding for ['562']:  67%|██████▋   | 37/55 [01:11<00:31,  1.78s/it]

Skipping concept pair C.OBrian, M.OBrian: Insufficient samples for C.OBrian vs M.OBrian


Decoding for ['562']:  89%|████████▉ | 49/55 [01:35<00:10,  1.76s/it]

Skipping concept pair K.Hayes, T.Lennox: Insufficient samples for K.Hayes vs T.Lennox


Decoding for ['562']: 100%|██████████| 55/55 [01:48<00:00,  1.96s/it]
